### category 8 : sandalwood (top, middle, base)
#### ==> cedar로 타겟 향 변경 및 타겟 note 종류도  middle, base로 변경

#### 순서
1. 향수 정보 열기
2. <카테고리  3> : top_notes에 'rose'를 포함하는 향수만 필터링
3. top_notes 리스트에의 향들 : 카테고리 3에 포함되는 향 갯수, 다음 가까운 카테고리에 포함되는 향 갯수, 그 다음...
4. 각 갯수 * 해당 가중치(1, 0.7, 0.4) ==> 'score'
5. score에 따라서 정렬 후 top 3 향수 추출
6. 저장
*** 
* 변경된 사항 : 타겟 향과 타겟 노트 종류, 그리고 score의 평균 값으로 추천 향수 추출

In [1]:
import pandas as pd
from ast import literal_eval

import warnings
warnings.filterwarnings("ignore")

In [99]:
# 향수 데이터를 필요한 칼럼만 남겨 perfume_df에 불러오기
perfume_df = pd.read_csv('../data/1st treatment_perfume.csv', index_col = 0)
perfume_df = perfume_df[['pk', 'name', 'top_notes', 'heart_notes', 'base_notes']]

perfume_df.head(2)

,pk,name,top_notes,heart_notes,base_notes
0,26120003,X-Centric,"[178, 224, 229, 292, 388, 395, 646]","[358, 545, 785]","[26, 193, 624, 692]"
1,26120006,GFT,"[96, 515, 562]","[292, 512, 886]","[193, 624, 649]"


In [100]:
# 각 카테고리별 향들의 정보가 들어있는 cate_note.csv불러오기
cat_notes = pd.read_csv('../data/cate_note.csv', index_col = 0)
cat_notes

,category,notes,note_idx
category_idx,,,
1,citrus_smells,"['bergamot', 'bitter orange', 'blood orange', ...","[96, 101, 114, 155, 213, 226, 227, 233, 346, 3..."
2,fruits_vegetables_and_nuts,"['acai berry', 'acerola', 'almond', 'apple', '...","[3, 4, 16, 45, 47, 49, 51, 56, 57, 75, 86, 90,..."
3,flowers,"['acacia', 'acerola blossom', 'aglaia', 'almon...","[2, 5, 10, 17, 21, 22, 25, 40, 41, 46, 52, 64,..."
4,white_flowers,"['arum lily', 'black locust', 'boronia', 'cari...","[61, 106, 122, 179, 302, 357, 367, 389, 437, 4..."
5,greens_herbs_and_fougeres,"['agave', 'algae', 'aloe vera', 'ammophila (be...","[8, 14, 19, 37, 42, 48, 59, 60, 65, 69, 70, 77..."
6,spices,"['allspice', 'anise', 'asafoetida', 'bay leaf'...","[15, 44, 62, 84, 151, 177, 178, 189, 222, 224,..."
7,sweets_and_gourmand_smells,"['acetyl furan', 'agave nectar', 'biscuit', 'b...","[6, 9, 100, 129, 133, 136, 137, 146, 148, 165,..."
8,woods_and_mosses,"['agarwood (oud)', 'akigalawood', 'alder', 'al...","[7, 11, 13, 18, 23, 39, 50, 54, 55, 58, 66, 74..."
9,resins_and_balsams,"['amberwood', 'balsamic vinegar', 'benzoin', '...","[30, 73, 95, 99, 131, 144, 153, 217, 218, 219,..."


In [101]:
# perfume_df의 note들 정보가 리스트가 아닌 string으로 저장되어 있으므로 리스트로 변환
# convert str to list
note_types = ['top_notes', 'heart_notes', 'base_notes']
for n_type in note_types:
    perfume_df[n_type] = perfume_df[n_type].apply(literal_eval)

In [102]:
# note에 대한 정보 (이름과  pk)을 가진 데이터 프레임
note8 = pd.read_csv('../data/concat_notes.csv')
note8 = note8[['pk', 'name']]
note8[note8['name'] == 'sandalwood'] # sandalwood pk값 확인

,pk,name
807,808,sandalwood


***

### sandalwood(808) 계산했던 코드

In [83]:
# top_notes에 808번 향을 포함하는 향수의 pk값을 top_808 리스트에 넣기
top_808 = []
for i in range(len(perfume_df)):
    for j in range(len(perfume_df['top_notes'][i])):
        if perfume_df['top_notes'][i][j] == 808:
            top_808.append(perfume_df['pk'][i])
top_808

[26120141,
 26120236,
 26120386,
 26120722,
 26120849,
 26120940,
 26120974,
 26121050,
 26121059,
 26121076,
 26121335,
 26121375,
 26121388,
 26121413,
 26121431,
 26121442,
 26121447,
 26121452,
 26121460,
 26121471,
 26121511,
 26121603,
 26121665,
 26121688,
 26121739,
 26121817,
 26122037,
 26122068,
 26122269,
 26122372,
 26122408,
 26122426,
 26122434,
 26122570,
 26122750,
 26122759,
 26122808,
 26122809,
 26123012,
 26123027,
 26123187,
 26123204,
 26123254,
 26123255,
 26123422,
 26123440,
 26123450,
 26123462,
 26123493,
 26123512,
 26123513,
 26123514,
 26123544,
 26123632,
 26123748,
 26123766,
 26124201,
 26124405,
 26124483,
 26124491,
 26124684,
 26124718,
 26124907,
 26124909,
 26124910,
 26124920,
 26124921,
 26124929,
 26124945,
 26125086,
 26125087,
 26125270,
 26125342,
 26125374,
 26125541,
 26125636,
 26125673,
 26125710,
 26125727,
 26125947,
 26125989,
 26126018,
 26126034,
 26126040,
 26126186,
 26126187,
 26126230,
 26126266,
 26126326,
 26126485,
 26126709,

In [88]:
# top_808리스트 속의 값을 pk로 가지는 향수만 골라서 새로 데이터 프레임 만들기
test = []
for i in top_808:
    test.append(perfume_df[perfume_df['pk'] == i])

df = test[0]
for i in range(1,len(test)):
    df = df.append(test[i])
df = df.reset_index(drop = True)
df

,pk,name,top_notes,heart_notes,base_notes
0,26120141,Astor,"[177, 808]",[],[]
1,26120236,Bois Precieux,"[26, 649, 808, 920]",[],[]
2,26120386,Comme des Garçons Parfum,"[193, 224, 259, 433, 708, 785, 808, 863]",[],[]
3,26120722,Greenbriar,"[26, 60, 512, 802, 808, 896, 924]",[],[]
4,26120849,Iris Silver Mist,"[27, 193, 237, 366, 472, 498, 624, 808, 924]",[],[]
...,...,...,...,...,...
874,26160383,Mon Oasis,"[193, 414, 473, 512, 624, 649, 698, 808, 900, ...",[],[]
875,26160427,Sandalo del Kashmir e Vetiver,"[376, 808, 924]",[],[]
876,26160491,Saffron Orris,"[562, 646, 669, 800, 802, 808]",[],[]
877,26160537,Tiwa,"[483, 530, 661, 692, 715, 808, 883, 924]",[],[]


filter된 향수 중 heart_note에 808가지는가

In [89]:
mid_808 = []

for i in range(len(df)):
    for j in range(len(df['heart_notes'][i])):
        if df['heart_notes'][i][j] == 808:
            mid_808.append(df['pk'][i])
print(mid_808[0:3])
print(len(mid_808))

[26135470, 26149730, 26149732]
3


In [90]:
test = []
for i in mid_808:
    test.append(perfume_df[perfume_df['pk'] == i])

df_middle = test[0]
for i in range(1,len(test)):
    df_middle = df_middle.append(test[i])
df_middle

,pk,name,top_notes,heart_notes,base_notes
4562,26135470,Traveler,"[403, 808]","[7, 193, 808, 924]","[26, 692, 908]"
9793,26149730,Black Bay : Capture,"[96, 229, 282, 808, 863]","[473, 808]","[193, 624, 920]"
9795,26149732,Oud Santal,"[229, 282, 808, 863]","[473, 808]",[920]


==> 3note들에 808이 다 들어가는게 없음

***

#### sandalwood에서 cedar로 타겟값 바꾸기

In [87]:
note8[note8['name'] == 'cedar'] # cedar의 pk값 확인

,pk,name
192,193,cedar


#### note값 바꾸고 middle & base만 보기

In [103]:
# heart_notes(middle note)에 타겟 향 (193)을 포함하는 향수 pk값 리스트로 반환
middle_193 = []

for i in range(len(perfume_df)):
    for j in range(len(perfume_df['heart_notes'][i])):
        if perfume_df['heart_notes'][i][j] == 193:
            middle_193.append(perfume_df['pk'][i])
print(middle_193[0:3])
print(len(middle_193))

[26120055, 26120057, 26120117]
592


In [104]:
# 반환된 리스트 값을 pk로 가지는 향수 정보를 보여주는 데이터 프레임
test = []
for i in middle_193:
    test.append(perfume_df[perfume_df['pk'] == i])

df = test[0]
for i in range(1,len(test)):
    df = df.append(test[i])
df =df.reset_index(drop = True)
df

,pk,name,top_notes,heart_notes,base_notes
0,26120055,Adidas Dynamic / Dynamic Pulse,"[227, 604]","[193, 353, 354]","[7, 908]"
1,26120057,Adidas Moves,"[44, 45, 604, 688, 708, 723, 906]","[193, 372, 376, 473, 480, 512, 624]","[649, 808, 896]"
2,26120117,Aqua Velva Ice Blue,"[96, 512, 515, 604, 715]","[193, 229, 473, 924]","[26, 498, 513, 624, 649]"
3,26120132,Arrogance pour Homme,"[96, 512, 708]","[193, 372, 433, 900]","[26, 498, 649, 692]"
4,26120149,Avon Uomo,"[82, 480]",[193],"[292, 624]"
...,...,...,...,...,...
587,26160415,Quai 3,"[529, 684]","[193, 599, 927]","[193, 229, 461, 649, 692]"
588,26160477,Luxor,"[178, 513]","[193, 472, 627, 900]","[293, 692]"
589,26160593,Junipera,"[114, 178, 481]","[193, 229, 513, 800]","[193, 624]"
590,26160605,No 34 : Aged Patchouli Attar - '1900 Vienna',"[96, 660, 661, 715, 765]","[193, 372, 473, 652]","[187, 417, 498, 669, 808, 920]"


In [106]:
# 필터링된 데이터 프레임에서 base_notes에 타겟 향을 가지는 향수의 pk값 리스트에 반환
base_193 = []

for i in range(len(df)):
    for j in range(len(df['base_notes'][i])):
        if df['base_notes'][i][j] == 193:
            base_193.append(df['pk'][i])
print(base_193[0:3])
print(len(base_193))

[26121714, 26126752, 26128579]
18


In [107]:
# 반환된 리스트의 값을 pk값으로 가지는 향수의 정보 데이터프레임으로 출력
test = []
for i in base_193:
    test.append(perfume_df[perfume_df['pk'] == i])

df_base = test[0]
for i in range(1,len(test)):
    df_base = df_base.append(test[i])
print(len(df_base))
df_final = df_base.reset_index(drop = True)

18


***

#### df_final을 점수 처리해서 top3 향수를 뽑자!
* category 8 = 1, cat1 = 0.7, cat5 = 0.3 점

In [108]:
df_final.head(3)

,pk,name,top_notes,heart_notes,base_notes
0,26121714,Xeryus Rouge,"[152, 497, 886]","[193, 708]","[193, 808]"
1,26126752,Next Generation,"[178, 388, 412, 715]","[193, 473]","[26, 193, 624, 692, 924]"
2,26128579,David Beckham Intense Instinct,"[376, 515, 593]","[193, 259]",[193]


#### 필터링이 된 df(df_final)에서 점수 계산 후 탑3개

In [109]:
cat8 = cat_notes['note_idx'].apply(literal_eval)[8] # 8번 카테고리에 해당하는 note들의 pk값 리스트
cat1 = cat_notes['note_idx'].apply(literal_eval)[1] # 1번 카테고리에 해당하는 note들의 pk값 리스트
cat5 = cat_notes['note_idx'].apply(literal_eval)[5] # 5번 카테고리에 해당하는 note들의 pk값 리스트

In [110]:
# 타겟 카테고리에 해당하는 향을 카테고리에 따라 a 또는 b 또는 c로 변경하는 함수
def convert_note(target_type):
    for i in range(len(df_final)):
        for j in range(len(df_final[target_type][i])):
            if df_final[target_type][i][j] in cat8:
                df_final[target_type][i][j] = 'a'
            elif df_final[target_type][i][j] in cat1:
                df_final[target_type][i][j] = 'b'
            elif df_final[target_type][i][j] in cat5:
                df_final[target_type][i][j] = 'c'
            else:
                pass

In [113]:
# target_type을 *_note 열들로 지정하고 위 조건에 해당하는 값을 변경
target_type = ['top_notes', 'heart_notes', 'base_notes']
for i in target_type:
    convert_note(i)
    
df_final.head(3)

,pk,name,top_notes,heart_notes,base_notes
0,26121714,Xeryus Rouge,"[c, b, c]","[a, 708]","[a, a]"
1,26126752,Next Generation,"[178, b, b, b]","[a, 473]","[26, a, 624, a, a]"
2,26128579,David Beckham Intense Instinct,"[376, b, 593]","[a, 259]",[a]


In [69]:
# Import Counter from collections
from collections import Counter

# 데이터 프레임에 score열을 만들고 위에서 번환된 note값에 따라 점수 계산
# 8번 카테고리 향은 1점, 1번 카테고리 향은 0.7점, 5번 카테고리 향은 0.4점

# 점수 계산 함수
def score_cal(target_type):
    score = Counter(df_final[target_type][i])['a'] * 1 + Counter(df_final[target_type][i])['b'] * 0.7 + Counter(df_final[target_type][i])['c'] * 0.4
    return score

df_final['score'] = 0
for i in range(len(df_final)):
    t_score = score_cal('top_notes')
    m_score = score_cal('heart_notes')
    b_score = score_cal('base_notes')
    df_final['score'][i] = t_score + m_score + b_score
df_final.head(3)

,pk,name,top_notes,heart_notes,base_notes,score
0,26121714,Xeryus Rouge,"[c, b, c]","[a, 708]","[a, a]",4.5
1,26126752,Next Generation,"[178, b, b, b]","[a, 473]","[26, a, 624, a, a]",6.1
2,26128579,David Beckham Intense Instinct,"[376, b, 593]","[a, 259]",[a],2.7
3,26132664,dirty ROSE,"[b, 725, 785, 785]","[a, 498, 652, 669, a, 785, 785, a]","[28, a, 513, a, 863, c, a]",7.1
4,26135681,Koh-Do,"[b, 472]","[a, 102, a, 246, 646]","[a, a, 908]",4.7


In [116]:
# score값에 따른 top 3
df_final.sort_values(by = 'score', ascending = False)[0:3]

,pk,name,top_notes,heart_notes,base_notes,score
6,26141692,Extrait de Cedrat,"[c, b, b]","[a, b, a]","[a, a, a, a, c]",8.9
3,26132664,dirty ROSE,"[b, 725, 785, 785]","[a, 498, 652, 669, a, 785, 785, a]","[28, a, 513, a, 863, c, a]",7.1
10,26146550,Creative Collection: 5Elements,"[44, 290, 372, 529, 593, 646, 708]","[a, 473, a, 785]","[a, 26, a, a, a, a, 920]",7.0


In [120]:
# 해당 향수의 전체 노트의 수 = 'total_notes'
df_final['total_notes'] = None
# 해당 향수의 평균 점수
df_final['score_avg'] = 0

# 노트들의 리스트 길이를 이용하여 평균 점수 구하기
for i in range(len(df_final)):
    df_final['total_notes'][i] = len(df_final['top_notes'][i] + df_final['heart_notes'][i] + df_final['base_notes'][i])
    df_final['score_avg'][i] = df_final['score'][i]/df_final['total_notes'][i]
df_final.head(3)

,pk,name,top_notes,heart_notes,base_notes,score,total_notes,score_avg
0,26121714,Xeryus Rouge,"[c, b, c]","[a, 708]","[a, a]",4.5,7,0.642857
1,26126752,Next Generation,"[178, b, b, b]","[a, 473]","[26, a, 624, a, a]",6.1,11,0.554545
2,26128579,David Beckham Intense Instinct,"[376, b, 593]","[a, 259]",[a],2.7,6,0.450000


In [121]:
# 평균 점수로 정한 top 3향수
df_final.sort_values(by = 'score_avg', ascending = False)[0:3]

,pk,name,top_notes,heart_notes,base_notes,score,total_notes,score_avg
6,26141692,Extrait de Cedrat,"[c, b, b]","[a, b, a]","[a, a, a, a, c]",8.9,11,0.809091
7,26143465,Bois d'Oud,"[a, b]","[a, 800]","[a, a, a, c]",6.1,8,0.762500
0,26121714,Xeryus Rouge,"[c, b, c]","[a, 708]","[a, a]",4.5,7,0.642857


In [137]:
df_base = pd.read_csv('../data/fill_season_catagory.csv')

# 점수 계산 후 나온 상위 3개의 향수의 pk값을 이용하여 향수 최종 데이터에서 동일 pk값을 가진 향수 정보를 추출 
result = df_final.sort_values(by = 'score_avg', ascending = False)[0:3]
result_pk = result['pk'].reset_index(drop=True)

fin = df_base[df_base['pk'] == result_pk[0]]
for i in range(1, len(result_pk)):
    fin = fin.append(df_base[df_base['pk'] == result_pk[i]])

# 결과 데이터 프레임 csv로 저장
fin.to_csv('../data/category8_result.csv')

0    26141692
1    26143465
2    26121714
Name: pk, dtype: int64

***

#### 헷갈려서 다른 거 구하고 있던 코드
* 특정 향을 가진 향수가 아닌 특정 카테고리내의 향을 포함한 향수를 구하고 있었다

In [25]:
cat8 = cat_notes['note_idx'].apply(literal_eval)[8] # 8번 카테고리에 해당하는 note들의 pk값 리스트
middle_perfume = []

for i in range(len(perfume_df)):
    for j in range(len(perfume_df['heart_notes'][i])):
        if perfume_df['heart_notes'][i][j] in cat8:
            top_perfume.append(perfume_df['pk'][i])
top_perfume

[26120003,
 26120019,
 26120076,
 26120105,
 26120141,
 26120159,
 26120223,
 26120232,
 26120235,
 26120236,
 26120236,
 26120244,
 26120263,
 26120365,
 26120384,
 26120384,
 26120384,
 26120386,
 26120386,
 26120387,
 26120387,
 26120418,
 26120418,
 26120467,
 26120467,
 26120488,
 26120504,
 26120504,
 26120519,
 26120519,
 26120519,
 26120528,
 26120528,
 26120538,
 26120546,
 26120555,
 26120573,
 26120573,
 26120586,
 26120586,
 26120613,
 26120613,
 26120695,
 26120722,
 26120722,
 26120745,
 26120796,
 26120806,
 26120808,
 26120849,
 26120849,
 26120849,
 26120888,
 26120927,
 26120933,
 26120940,
 26120974,
 26121044,
 26121050,
 26121059,
 26121059,
 26121059,
 26121059,
 26121076,
 26121076,
 26121095,
 26121095,
 26121107,
 26121113,
 26121113,
 26121114,
 26121137,
 26121137,
 26121143,
 26121144,
 26121144,
 26121146,
 26121199,
 26121199,
 26121210,
 26121210,
 26121210,
 26121262,
 26121266,
 26121272,
 26121272,
 26121335,
 26121335,
 26121341,
 26121341,
 26121341,

In [26]:
test = []
for i in top_perfume:
    test.append(perfume_df[perfume_df['pk'] == i])

df = test[0]
for i in range(1,len(test)):
    df = df.append(test[i])
df

,pk,name,top_notes,heart_notes,base_notes
0,26120003,X-Centric,"[178, 224, 229, 292, 388, 395, 646]","[358, 545, 785]","[26, 193, 624, 692]"
5,26120019,1881 pour Homme,"[82, 96, 237, 292, 481, 512, 515, 793, 886]","[102, 785, 966]","[26, 808]"
32,26120076,Alain Delon Pour Homme,"[82, 178, 292, 604, 883]","[96, 473, 785]","[26, 624, 808, 920, 924]"
42,26120105,Antaeus,"[96, 193, 229, 259, 515, 530, 628]","[82, 473, 669, 692, 785, 808, 896]","[89, 191, 498, 513, 624, 649, 652]"
60,26120141,Astor,"[177, 808]",[],[]
...,...,...,...,...,...
14224,26160623,Casino Elixir,"[16, 26, 28, 30, 45, 96, 98, 102, 193, 473, 64...",[],[]
14224,26160623,Casino Elixir,"[16, 26, 28, 30, 45, 96, 98, 102, 193, 473, 64...",[],[]
14232,26160637,Gardenia & Oud Absolu,"[7, 26, 367, 624]",[],[]
14233,26160638,Confetti,"[785, 808, 928]",[],[]


In [27]:
df = df.reset_index(drop=True)

In [34]:
df[df['pk'] == 26120105]

,pk,name,top_notes,heart_notes,base_notes
3,26120105,Antaeus,"[96, 193, 229, 259, 515, 530, 628]","[82, 473, 669, 692, 785, 808, 896]","[89, 191, 498, 513, 624, 649, 652]"
3,26120105,Antaeus,"[96, 193, 229, 259, 515, 530, 628]","[82, 473, 669, 692, 785, 808, 896]","[89, 191, 498, 513, 624, 649, 652]"


In [29]:
cat8 = cat_notes['note_idx'].apply(literal_eval)[8]
middle_perfume = []

for i in range(len(df)):
    for j in range(len(df['heart_notes'][i])):
        if df['heart_notes'][i][j] in cat8:
            middle_perfume.append(df['pk'][i])
len(middle_perfume)

216

In [33]:
middle_perfume

[26120105,
 26120105,
 26120159,
 26120223,
 26120504,
 26120504,
 26120504,
 26120504,
 26120586,
 26120586,
 26120695,
 26120745,
 26120745,
 26120745,
 26120796,
 26120933,
 26120933,
 26121107,
 26121372,
 26121372,
 26121372,
 26121406,
 26121406,
 26121414,
 26122468,
 26122489,
 26122742,
 26122742,
 26123541,
 26123577,
 26124394,
 26124404,
 26124404,
 26124455,
 26126156,
 26128362,
 26128362,
 26128460,
 26128460,
 26128460,
 26128571,
 26128725,
 26128725,
 26128725,
 26129336,
 26129336,
 26129336,
 26129336,
 26129666,
 26132164,
 26133124,
 26133266,
 26133676,
 26133933,
 26134549,
 26134803,
 26134803,
 26134834,
 26134834,
 26135031,
 26135192,
 26135192,
 26135192,
 26135470,
 26135470,
 26135470,
 26135470,
 26135470,
 26135470,
 26135470,
 26135470,
 26135652,
 26135845,
 26135863,
 26136672,
 26136672,
 26136674,
 26136730,
 26136730,
 26136730,
 26136765,
 26137001,
 26137192,
 26137192,
 26137778,
 26137778,
 26137778,
 26137954,
 26137954,
 26138040,
 26138040,

In [30]:
test = []
for i in middle_perfume:
    test.append(df[df['pk'] == i])

df = test[0]
for i in range(1,len(test)):
    df = df.append(test[i])
df

,pk,name,top_notes,heart_notes,base_notes
3,26120105,Antaeus,"[96, 193, 229, 259, 515, 530, 628]","[82, 473, 669, 692, 785, 808, 896]","[89, 191, 498, 513, 624, 649, 652]"
3,26120105,Antaeus,"[96, 193, 229, 259, 515, 530, 628]","[82, 473, 669, 692, 785, 808, 896]","[89, 191, 498, 513, 624, 649, 652]"
5,26120159,Balafre,"[96, 292, 512, 639]","[180, 193, 341, 372, 802]","[26, 513, 624, 649, 924]"
6,26120223,Body Kouros,"[336, 472]","[193, 552]",[]
26,26120504,Eau de Grey Flannel,"[193, 292, 562, 896]","[229, 336, 512, 692]","[808, 924]"
...,...,...,...,...,...
4337,26159952,Oud Divinity,"[7, 708, 785]","[178, 187, 293, 920]","[26, 513, 692, 808, 908]"
4344,26159987,Secret Love,"[7, 455, 785, 800]","[187, 473, 513, 692, 920]","[26, 403, 624, 808]"
4426,26160397,Black Tea Musk,"[161, 808]","[193, 472, 652, 900]","[193, 341, 472]"
4428,26160415,Quai 3,"[529, 684]","[193, 599, 927]","[193, 229, 461, 649, 692]"
